# How Player Height Affects the NBA

## Analysis Team
*Lindsay Reynolds, Nick Sheets*

## Data Exploration and Cleanup
### Data Exploration
####  Resources
Identifying resources to use was difficult for this project.
* The early issues were in locating an API that could be used.
  * APIs with too few calls a day for free accounts  
  * An API with free trial, but to be used to make a decision about subscribing
  * Finding others describing the APIs on www.nba.com as not well documented  
  * A well-documented API that could have been pursued with more time
    * [pip install nba-api](https://pypi.org/project/nba-api/)  
* Abandoned looking for an API to use and switched to searching for resources on Kaggle
  * [Kaggle: Basketball Dataset](https://www.kaggle.com/wyattowalsh/basketball) - Wyatt Walsh  
    * Several great csv files, including player heights, stats and salaries,  
      but download of the project is in SQLite rather than for individual csv files
  * [Kaggle: NBA Games Data](https://www.kaggle.com/nathanlauga/nba-games) - Nathan Lauga
    * Not much available for player profile (no heights, no background details)
    * games_details.csv 
      * There are > 600,000 rows of data for each player, for each game
      * Game ID instead of dates
      * Complexity of Game ID (preseason start with 1, regular season 2 and post-season 4)
      * Issue would still remain to sum up every player in the data set for each season
  * [Kaggle: NBA Players Data](https://www.kaggle.com/justinas/nba-players-data) - Justinas Cirtautas
    * Find heights of players here
    * Find yearly averages by player for points, rebounds and assists
    * Limitation - there is no data for blocks
    * Decision made to proceed using this data
* To analyze salary found 2020-21 salaries online  
  [Sports Reference LLC:  Basketball-Reference.com "2020-21 NBA Player Contracts" (4/22/2021)](https://www.basketball-reference.com/contracts/players.html) 
    
####  Data Cleanup
Filtered down to only the seasons desired, then filtered more for the columns needed  
Height started in cm and needed converted to inches (and then to feet and inches as needed)  
Needed to sync up how the player column was named in each df
* One issue was with salary df, where Player is like Stephen Curry\curryst01


#### Insights during Data Exploration and Cleanup
In wanting to examine salary
* Data is for the 2020-21 season and forward
* Surpising drop in the players being analyzed for salary  
  * From the player data being analyzed for stats across three seasons  
    (with most recent season 2018-19), we had 748 different players  
   * 2020-21 Salaries started as 508 rows, with 488 different players
   * Of the 748 players for stats analysis, only 324 of those have 2020-21 salaries
   * Over half of NBA players from just a few years ago are not playing in NBA today    

In [1]:
# Set up dependencies and read in csv files needed
import os
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as img

In [2]:
csv_path = os.path.join("Resources", "all_seasons.csv")
csv_path

'Resources\\all_seasons.csv'

In [3]:
salary_path = os.path.join("Resources", "NBA_salaries.csv")
salary_path

'Resources\\NBA_salaries.csv'

In [4]:
# Create df for player info/stats
nba_data_set_df = pd.read_csv(csv_path)
nba_data_set_df

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Dennis Rodman,CHI,36,198.12,99.790240,Southeastern Oklahoma State,USA,1986,2,...,5.7,16.1,3.1,16.1,0.186,0.323,0.100,0.479,0.113,1996-97
1,1,Dwayne Schintzius,LAC,28,215.90,117.933920,Florida,USA,1990,1,...,2.3,1.5,0.3,12.3,0.078,0.151,0.175,0.430,0.048,1996-97
2,2,Earl Cureton,TOR,39,205.74,95.254320,Detroit Mercy,USA,1979,3,...,0.8,1.0,0.4,-2.1,0.105,0.102,0.103,0.376,0.148,1996-97
3,3,Ed O'Bannon,DAL,24,203.20,100.697424,UCLA,USA,1995,1,...,3.7,2.3,0.6,-8.7,0.060,0.149,0.167,0.399,0.077,1996-97
4,4,Ed Pinckney,MIA,34,205.74,108.862080,Villanova,USA,1985,1,...,2.4,2.4,0.2,-11.2,0.109,0.179,0.127,0.611,0.040,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11140,11140,Maxi Kleber,DAL,28,208.28,108.862080,None,Germany,Undrafted,Undrafted,...,9.1,5.4,1.1,4.6,0.056,0.140,0.136,0.605,0.064,2019-20
11141,11141,Melvin Frazier Jr.,ORL,23,195.58,97.522280,Tulane,USA,2018,2,...,1.2,0.3,0.1,-2.4,0.018,0.058,0.164,0.480,0.033,2019-20
11142,11142,Meyers Leonard,MIA,28,213.36,117.933920,Illinois,USA,2012,1,...,6.1,5.1,1.1,5.6,0.029,0.217,0.120,0.640,0.076,2019-20
11143,11143,Norvel Pelle,PHI,27,208.28,104.779752,None,USA,Undrafted,Undrafted,...,2.1,3.0,0.4,-16.4,0.085,0.237,0.126,0.521,0.056,2019-20


In [5]:
# Create df from the salaries csv
nba_salary_df = pd.read_csv(salary_path)
nba_salary_df

,Rk,Player,Tm,2020-21,2021-22,2022-23,2023-24,2024-25,2025-26,Signed Using,Guaranteed
0,1,Stephen Curry\curryst01,GSW,43006362.0,45780966.0,NaN,NaN,NaN,NaN,Bird Rights,88787328.0
1,2,Russell Westbrook\westbru01,WAS,41358814.0,44211146.0,47063478.0,NaN,NaN,NaN,Bird Rights,85569960.0
2,3,Chris Paul\paulch01,PHO,41358814.0,44211146.0,NaN,NaN,NaN,NaN,Bird Rights,41358814.0
3,4,John Wall\walljo01,HOU,41254920.0,44310840.0,47366760.0,NaN,NaN,NaN,Bird Rights,85565760.0
4,5,James Harden\hardeja01,BRK,40824000.0,43848000.0,46872000.0,NaN,NaN,NaN,Bird Rights,84672000.0
...,...,...,...,...,...,...,...,...,...,...,...
503,504,Justin Anderson\anderju01,PHI,125000.0,NaN,NaN,NaN,NaN,NaN,NaN,125000.0
504,505,Troy Williams\willitr02,HOU,122741.0,122741.0,122741.0,NaN,NaN,NaN,NaN,368223.0
505,506,Demetrius Jackson\jacksde01,BOS,92857.0,92857.0,92857.0,92857.0,NaN,NaN,NaN,371428.0
506,507,Henry Ellenson\ellenhe01,TOR,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,50000.0


In [6]:
# Adjust Player column to only show the first and last name
nba_salary_df["Player"] = nba_salary_df["Player"].str.split("\\").str[0]
nba_salary_df

,Rk,Player,Tm,2020-21,2021-22,2022-23,2023-24,2024-25,2025-26,Signed Using,Guaranteed
0,1,Stephen Curry,GSW,43006362.0,45780966.0,NaN,NaN,NaN,NaN,Bird Rights,88787328.0
1,2,Russell Westbrook,WAS,41358814.0,44211146.0,47063478.0,NaN,NaN,NaN,Bird Rights,85569960.0
2,3,Chris Paul,PHO,41358814.0,44211146.0,NaN,NaN,NaN,NaN,Bird Rights,41358814.0
3,4,John Wall,HOU,41254920.0,44310840.0,47366760.0,NaN,NaN,NaN,Bird Rights,85565760.0
4,5,James Harden,BRK,40824000.0,43848000.0,46872000.0,NaN,NaN,NaN,Bird Rights,84672000.0
...,...,...,...,...,...,...,...,...,...,...,...
503,504,Justin Anderson,PHI,125000.0,NaN,NaN,NaN,NaN,NaN,NaN,125000.0
504,505,Troy Williams,HOU,122741.0,122741.0,122741.0,NaN,NaN,NaN,NaN,368223.0
505,506,Demetrius Jackson,BOS,92857.0,92857.0,92857.0,92857.0,NaN,NaN,NaN,371428.0
506,507,Henry Ellenson,TOR,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,50000.0


In [7]:
# Remove columns not needed from salary df and rename player column like the stats df
salary_data_filter_df = nba_salary_df[["Player", "2020-21"]]
salary_data_filter_df = salary_data_filter_df.rename(columns={"Player" : "player_name"})
salary_data_filter_df

,player_name,2020-21
0,Stephen Curry,43006362.0
1,Russell Westbrook,41358814.0
2,Chris Paul,41358814.0
3,John Wall,41254920.0
4,James Harden,40824000.0
...,...,...
503,Justin Anderson,125000.0
504,Troy Williams,122741.0
505,Demetrius Jackson,92857.0
506,Henry Ellenson,50000.0


In [8]:
# Examine the unique player names for those with 2020-21 salaries
players_with_2021_sal = salary_data_filter_df["player_name"].unique()
print(len(players_with_2021_sal))
players_with_2021_sal

488


array(['Stephen Curry', 'Russell Westbrook', 'Chris Paul', 'John Wall',
       'James Harden', 'LeBron James', 'Kevin Durant', 'Paul George',
       'Klay Thompson', 'Mike Conley', 'Jimmy Butler', 'Kemba Walker',
       'Kawhi Leonard', 'Tobias Harris', 'Kyrie Irving',
       'Khris Middleton', 'Anthony Davis', 'Blake Griffin',
       'Damian Lillard', 'Kevin Love', 'Kyle Lowry', 'Joel Embiid',
       'Andrew Wiggins', 'Kristaps Porziņģis', 'Devin Booker',
       'Karl-Anthony Towns', 'CJ McCollum', 'Ben Simmons', 'Jamal Murray',
       'Pascal Siakam', 'Bradley Beal', 'Andre Drummond',
       "D'Angelo Russell", 'Nikola Jokić', 'Gordon Hayward',
       'Otto Porter', 'DeMar DeRozan', 'Giannis Antetokounmpo',
       'Steven Adams', 'Al Horford', 'Brandon Ingram', 'Rudy Gobert',
       'Jrue Holiday', 'Nikola Vučević', 'Buddy Hield', 'Jaylen Brown',
       'Draymond Green', 'Harrison Barnes', 'Fred VanVleet',
       'Victor Oladipo', 'Malcolm Brogdon', 'Danilo Gallinari',
       'Zach L

In [9]:
# For stats filter down to only the seasons to analyze
# Choosing three most recent seasons with complete season data in the csv 
seasons_needed = ["2016-17", "2017-18", "2018-19"]
filter_data_set_df = nba_data_set_df[nba_data_set_df["season"].isin(seasons_needed)]
filter_data_set_df

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
9075,9075,Anthony Bennett,BKN,24,203.20,106.594120,Nevada-Las Vegas,Canada,2013,1,...,5.0,3.4,0.5,-10.6,0.104,0.207,0.191,0.533,0.075,2016-17
9076,9076,Anthony Brown,ORL,24,200.66,95.707912,Stanford,USA,2015,2,...,3.9,3.0,0.7,1.3,0.048,0.163,0.158,0.430,0.080,2016-17
9077,9077,Anthony Davis,NOP,24,210.82,114.758776,Kentucky,USA,2012,1,...,28.0,11.8,2.1,1.7,0.067,0.269,0.326,0.580,0.110,2016-17
9078,9078,Andrew Wiggins,MIN,22,203.20,90.264808,Kansas,Canada,2014,1,...,23.6,4.0,2.3,-0.5,0.039,0.087,0.288,0.534,0.104,2016-17
9079,9079,Andrew Nicholson,BKN,27,205.74,113.398000,St. Bonaventure,Canada,2012,1,...,2.6,1.6,0.3,-17.6,0.050,0.156,0.178,0.427,0.047,2016-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10626,10626,Abdel Nader,OKC,25,198.12,102.058200,Iowa State,Egypt,2016,2,...,4.0,1.9,0.3,-9.5,0.017,0.139,0.148,0.522,0.044,2018-19
10627,10627,Aaron Holiday,IND,22,185.42,83.914520,UCLA,USA,2018,1,...,5.9,1.3,1.7,7.0,0.008,0.088,0.206,0.518,0.180,2018-19
10628,10628,Aaron Gordon,ORL,23,205.74,99.790240,Arizona,USA,2014,1,...,16.0,7.4,3.7,1.5,0.047,0.165,0.213,0.538,0.166,2018-19
10629,10629,Alec Burks,SAC,27,198.12,97.068688,Colorado,USA,2011,1,...,8.8,3.7,2.0,-11.1,0.021,0.151,0.185,0.523,0.142,2018-19


In [10]:
pd.options.display.float_format = '{:,.2f}'.format

In [11]:
# Filter the df down to the desired columns
stats_needed_df = filter_data_set_df[["season", "player_name", "player_height", "pts", "reb", "ast", "team_abbreviation", "college", "country"]]
stats_needed_df

,season,player_name,player_height,pts,reb,ast,team_abbreviation,college,country
9075,2016-17,Anthony Bennett,203.20,5.00,3.40,0.50,BKN,Nevada-Las Vegas,Canada
9076,2016-17,Anthony Brown,200.66,3.90,3.00,0.70,ORL,Stanford,USA
9077,2016-17,Anthony Davis,210.82,28.00,11.80,2.10,NOP,Kentucky,USA
9078,2016-17,Andrew Wiggins,203.20,23.60,4.00,2.30,MIN,Kansas,Canada
9079,2016-17,Andrew Nicholson,205.74,2.60,1.60,0.30,BKN,St. Bonaventure,Canada
...,...,...,...,...,...,...,...,...,...
10626,2018-19,Abdel Nader,198.12,4.00,1.90,0.30,OKC,Iowa State,Egypt
10627,2018-19,Aaron Holiday,185.42,5.90,1.30,1.70,IND,UCLA,USA
10628,2018-19,Aaron Gordon,205.74,16.00,7.40,3.70,ORL,Arizona,USA
10629,2018-19,Alec Burks,198.12,8.80,3.70,2.00,SAC,Colorado,USA


In [12]:
# Identify unique number of players across the three seasons examined
players = stats_needed_df["player_name"].unique()
print(len(players))
players

748


array(['Anthony Bennett', 'Anthony Brown', 'Anthony Davis',
       'Andrew Wiggins', 'Andrew Nicholson', 'Andrew Harrison',
       'Andrew Bogut', 'Amir Johnson', 'Andre Iguodala', 'Andre Drummond',
       'Anderson Varejao', 'Alonzo Gee', 'Allen Crabbe', 'Alexis Ajinca',
       'Anthony Morrow', 'Alex Poythress', 'Alex Len', 'Andre Roberson',
       'Anthony Tolliver', 'Boban Marjanovic', 'Arinze Onuaku',
       'Brandon Ingram', 'Alex Abrines', 'Brandon Bass', 'Brandan Wright',
       'Bradley Beal', 'Boris Diaw', 'Bojan Bogdanovic', 'Bobby Portis',
       'Archie Goodwin', 'Bobby Brown', 'Beno Udrih', 'Ben McLemore',
       'Ben Bentil', 'Axel Toupane', 'Avery Bradley', 'Austin Rivers',
       'Arron Afflalo', 'Aron Baynes', 'Blake Griffin', 'Alec Burks',
       'Vince Carter', 'Alan Anderson', 'Tyus Jones', 'Udonis Haslem',
       'Victor Oladipo', 'Wade Baldwin IV', 'Wayne Ellington',
       'Wayne Selden', 'Wesley Johnson', 'Wesley Matthews',
       'Tyson Chandler', 'Will Barton

In [13]:
# Want to add a calculated column for height in inches (2.54 cm in an inch)

# Label the current player_height column as cm and add column for height in inches 
stats_with_col_rename_df = stats_needed_df.rename(columns={"player_height":"height (cm)", "team_abbreviation":"team"})
height = stats_with_col_rename_df['height (cm)'] / 2.54
stats_with_col_rename_df["height (in)"] = height

# Reorganize and sort columns
organized_stats_df = stats_with_col_rename_df[["season", "player_name","height (in)","height (cm)","pts","reb","ast","college","country", "team"]]
organized_stats_df = organized_stats_df.sort_values(by=['player_name'])

# View the results
organized_stats_df

,season,player_name,height (in),height (cm),pts,reb,ast,college,country,team
9147,2016-17,AJ Hammons,84.00,213.36,2.20,1.60,0.20,Purdue,USA,DAL
9647,2017-18,Aaron Brooks,72.00,182.88,2.30,0.50,0.60,Oregon,USA,MIN
9146,2016-17,Aaron Brooks,72.00,182.88,5.00,1.10,1.90,Oregon,USA,IND
10628,2018-19,Aaron Gordon,81.00,205.74,16.00,7.40,3.70,Arizona,USA,ORL
9145,2016-17,Aaron Gordon,81.00,205.74,12.70,5.10,1.90,Arizona,USA,ORL
...,...,...,...,...,...,...,...,...,...,...
9862,2017-18,Zaza Pachulia,83.00,210.82,5.40,4.70,1.60,None,Georgia,GSW
10455,2018-19,Zaza Pachulia,83.00,210.82,3.90,3.90,1.30,None,Georgia,DET
10630,2018-19,Zhaire Smith,76.00,193.04,6.70,2.20,1.70,Texas Tech,USA,PHI
9861,2017-18,Zhou Qi,85.00,215.90,1.20,1.20,0.10,None,China,HOU


In [14]:
# Use the DataFrame to make a new one with only one line per player with personal info (no multi-year stats)
player_info_df = organized_stats_df[["player_name","height (in)","college","country", "team"]]

player_info_df

,player_name,height (in),college,country,team
9147,AJ Hammons,84.00,Purdue,USA,DAL
9647,Aaron Brooks,72.00,Oregon,USA,MIN
9146,Aaron Brooks,72.00,Oregon,USA,IND
10628,Aaron Gordon,81.00,Arizona,USA,ORL
9145,Aaron Gordon,81.00,Arizona,USA,ORL
...,...,...,...,...,...
9862,Zaza Pachulia,83.00,None,Georgia,GSW
10455,Zaza Pachulia,83.00,None,Georgia,DET
10630,Zhaire Smith,76.00,Texas Tech,USA,PHI
9861,Zhou Qi,85.00,None,China,HOU


In [15]:
# Drop duplicates so each player only appears once in a player_info_df
player_count = player_info_df['player_name'].unique()
unique_players_info_df = player_info_df.drop_duplicates(subset=["player_name"])

# Print length of the list of unique players across the three seasons to analyze and view df
print(len(player_count))
unique_players_info_df

748


,player_name,height (in),college,country,team
9147,AJ Hammons,84.00,Purdue,USA,DAL
9647,Aaron Brooks,72.00,Oregon,USA,MIN
10628,Aaron Gordon,81.00,Arizona,USA,ORL
9144,Aaron Harrison,78.00,Kentucky,USA,CHA
10627,Aaron Holiday,73.00,UCLA,USA,IND
...,...,...,...,...,...
10454,Zach Lofton,76.00,New Mexico State,USA,DET
9863,Zach Randolph,81.00,Michigan State,USA,SAC
9134,Zaza Pachulia,83.00,None,Georgia,GSW
10630,Zhaire Smith,76.00,Texas Tech,USA,PHI


In [16]:
# In organized_stats_df, for 3 seasons being analyzed each player has up to three rows 
# Group by player for calculating averages across the 3 seasons
player_groupby = organized_stats_df.groupby(["player_name"])
player_groupby.mean()

,height (in),height (cm),pts,reb,ast
player_name,,,,,
AJ Hammons,84.00,213.36,2.20,1.60,0.20
Aaron Brooks,72.00,182.88,3.65,0.80,1.25
Aaron Gordon,81.00,205.74,15.43,6.80,2.63
Aaron Harrison,78.00,198.12,3.45,1.65,0.90
Aaron Holiday,73.00,185.42,5.90,1.30,1.70
...,...,...,...,...,...
Zach Lofton,76.00,193.04,0.00,0.00,0.00
Zach Randolph,81.00,205.74,14.30,7.45,1.95
Zaza Pachulia,83.00,210.82,5.13,4.83,1.60


In [17]:
# Create a DataFrame from the group by
mean_groupby_df = pd.DataFrame(player_groupby.mean())
mean_groupby_df

,height (in),height (cm),pts,reb,ast
player_name,,,,,
AJ Hammons,84.00,213.36,2.20,1.60,0.20
Aaron Brooks,72.00,182.88,3.65,0.80,1.25
Aaron Gordon,81.00,205.74,15.43,6.80,2.63
Aaron Harrison,78.00,198.12,3.45,1.65,0.90
Aaron Holiday,73.00,185.42,5.90,1.30,1.70
...,...,...,...,...,...
Zach Lofton,76.00,193.04,0.00,0.00,0.00
Zach Randolph,81.00,205.74,14.30,7.45,1.95
Zaza Pachulia,83.00,210.82,5.13,4.83,1.60


In [18]:
# May need number of seasons being analyzed for each player (range from 1 to 3)
seasons = player_groupby.count()['season']
seasons

player_name
AJ Hammons        1
Aaron Brooks      2
Aaron Gordon      3
Aaron Harrison    2
Aaron Holiday     1
                 ..
Zach Lofton       1
Zach Randolph     2
Zaza Pachulia     3
Zhaire Smith      1
Zhou Qi           2
Name: season, Length: 748, dtype: int64

In [19]:
# Remove the unwanted column for height in cm
del mean_groupby_df['height (cm)']
mean_groupby_df

,height (in),pts,reb,ast
player_name,,,,
AJ Hammons,84.00,2.20,1.60,0.20
Aaron Brooks,72.00,3.65,0.80,1.25
Aaron Gordon,81.00,15.43,6.80,2.63
Aaron Harrison,78.00,3.45,1.65,0.90
Aaron Holiday,73.00,5.90,1.30,1.70
...,...,...,...,...
Zach Lofton,76.00,0.00,0.00,0.00
Zach Randolph,81.00,14.30,7.45,1.95
Zaza Pachulia,83.00,5.13,4.83,1.60


In [20]:
# Create a df with player info and the stats averages across seasons being analyzed
merged_player_info_df = pd.merge(mean_groupby_df, unique_players_info_df, on="player_name")
merged_player_info_df

,player_name,height (in)_x,pts,reb,ast,height (in)_y,college,country,team
0,AJ Hammons,84.00,2.20,1.60,0.20,84.00,Purdue,USA,DAL
1,Aaron Brooks,72.00,3.65,0.80,1.25,72.00,Oregon,USA,MIN
2,Aaron Gordon,81.00,15.43,6.80,2.63,81.00,Arizona,USA,ORL
3,Aaron Harrison,78.00,3.45,1.65,0.90,78.00,Kentucky,USA,CHA
4,Aaron Holiday,73.00,5.90,1.30,1.70,73.00,UCLA,USA,IND
...,...,...,...,...,...,...,...,...,...
743,Zach Lofton,76.00,0.00,0.00,0.00,76.00,New Mexico State,USA,DET
744,Zach Randolph,81.00,14.30,7.45,1.95,81.00,Michigan State,USA,SAC
745,Zaza Pachulia,83.00,5.13,4.83,1.60,83.00,None,Georgia,GSW
746,Zhaire Smith,76.00,6.70,2.20,1.70,76.00,Texas Tech,USA,PHI


In [21]:
# Remove duplicate column containing height
cleaned_merged_player_info_df = merged_player_info_df.drop("height (in)_y", 1)
cleaned_merged_player_info_df

,player_name,height (in)_x,pts,reb,ast,college,country,team
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND
...,...,...,...,...,...,...,...,...
743,Zach Lofton,76.00,0.00,0.00,0.00,New Mexico State,USA,DET
744,Zach Randolph,81.00,14.30,7.45,1.95,Michigan State,USA,SAC
745,Zaza Pachulia,83.00,5.13,4.83,1.60,None,Georgia,GSW
746,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI


In [22]:
# Rename the height column that remains
final_merged_player_info_df = cleaned_merged_player_info_df.rename(columns = {"height (in)_x":"height (in)"})
final_merged_player_info_df

,player_name,height (in),pts,reb,ast,college,country,team
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND
...,...,...,...,...,...,...,...,...
743,Zach Lofton,76.00,0.00,0.00,0.00,New Mexico State,USA,DET
744,Zach Randolph,81.00,14.30,7.45,1.95,Michigan State,USA,SAC
745,Zaza Pachulia,83.00,5.13,4.83,1.60,None,Georgia,GSW
746,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI


In [23]:
final_merged_player_info_df.describe()

,height (in),pts,reb,ast
count,748.00,748.00,748.00,748.00
mean,78.87,7.20,3.09,1.62
std,3.36,5.46,2.24,1.57
min,69.00,0.00,0.00,0.00
25%,76.00,3.30,1.50,0.60
50%,79.00,5.80,2.58,1.10
75%,81.00,9.47,3.95,2.10
max,87.00,31.87,15.13,10.47


In [24]:
# Merge two df so that salary column for 2020-21 can be displayed
final_nba_stats_df = pd.merge(final_merged_player_info_df, salary_data_filter_df, on="player_name", how = "left")
final_nba_stats_df

,player_name,height (in),pts,reb,ast,college,country,team,2020-21
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL,nan
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN,nan
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL,"18,136,364.00"
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA,nan
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND,"2,345,640.00"
...,...,...,...,...,...,...,...,...,...
761,Zach Lofton,76.00,0.00,0.00,0.00,New Mexico State,USA,DET,nan
762,Zach Randolph,81.00,14.30,7.45,1.95,Michigan State,USA,SAC,nan
763,Zaza Pachulia,83.00,5.13,4.83,1.60,None,Georgia,GSW,nan
764,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI,"3,204,600.00"


In [25]:
# Explore why 766 rows now following left join (suspect a trade)
# Note: team column displayed below will be from the stats df not the salary df
final_nba_stats_df.head(25)

,player_name,height (in),pts,reb,ast,college,country,team,2020-21
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL,nan
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN,nan
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL,"18,136,364.00"
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA,nan
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND,"2,345,640.00"
5,Aaron Jackson,75.00,8.00,3.00,1.00,None,USA,HOU,nan
6,Abdel Nader,78.00,3.50,1.70,0.40,Iowa State,Egypt,BOS,"1,752,950.00"
7,Adreian Payne,82.00,3.85,1.80,0.20,Michigan State,USA,ORL,nan
8,Al Horford,82.00,13.50,6.97,4.63,Florida,Dominican Republic,BOS,"27,500,000.00"
9,Al Jefferson,82.00,7.55,4.10,0.85,None,USA,IND,nan


In [26]:
# Some duplicate player rows identified following salary merge, drop duplicates
# Traced duplicates back to the csv file,played for ATL in our stats, but contracts in 2020-21 are with Tor and Was 
final_nba_stats_df = final_nba_stats_df.drop_duplicates(subset=["player_name"], keep="first")
final_nba_stats_df

,player_name,height (in),pts,reb,ast,college,country,team,2020-21
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL,nan
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN,nan
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL,"18,136,364.00"
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA,nan
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND,"2,345,640.00"
...,...,...,...,...,...,...,...,...,...
761,Zach Lofton,76.00,0.00,0.00,0.00,New Mexico State,USA,DET,nan
762,Zach Randolph,81.00,14.30,7.45,1.95,Michigan State,USA,SAC,nan
763,Zaza Pachulia,83.00,5.13,4.83,1.60,None,Georgia,GSW,nan
764,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI,"3,204,600.00"


In [27]:
# Create a second df to be used for salary analysis (remove NaN, no salary in 2020-21)
final_nba_stats_with_sal_df = final_nba_stats_df.dropna()
final_nba_stats_with_sal_df

,player_name,height (in),pts,reb,ast,college,country,team,2020-21
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL,"18,136,364.00"
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND,"2,345,640.00"
6,Abdel Nader,78.00,3.50,1.70,0.40,Iowa State,Egypt,BOS,"1,752,950.00"
8,Al Horford,82.00,13.50,6.97,4.63,Florida,Dominican Republic,BOS,"27,500,000.00"
10,Al-Farouq Aminu,81.00,9.13,7.50,1.37,Wake Forest,USA,POR,"9,720,900.00"
...,...,...,...,...,...,...,...,...,...
749,Willie Cauley-Stein,84.00,10.93,6.63,1.97,Kentucky,USA,SAC,"4,000,000.00"
758,Yuta Watanabe,81.00,2.60,2.10,0.50,George Washington,Japan,MEM,"321,893.00"
759,Zach Collins,84.00,5.50,3.75,0.85,Gonzaga,USA,POR,"5,406,255.00"
760,Zach LaVine,77.00,19.77,4.00,3.50,UCLA,USA,MIN,"19,500,000.00"


In [28]:
# Binning the heights
group_names = ["<= 6ft", "6'1\" - 6'3\"", "6'4\" - 6'6\"", "6'7\" - 6'9\"", "6'10\" - 7'", "> 7ft"]
bins = [0, 72, 75, 78, 81, 84, 87]
final_merged_player_info_df["Height Range"] = pd.cut(final_merged_player_info_df["height (in)"], bins, labels=group_names)
final_merged_player_info_df


,player_name,height (in),pts,reb,ast,college,country,team,Height Range
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL,"6'10"" - 7'"
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN,<= 6ft
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL,"6'7"" - 6'9"""
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA,"6'4"" - 6'6"""
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND,"6'1"" - 6'3"""
...,...,...,...,...,...,...,...,...,...
743,Zach Lofton,76.00,0.00,0.00,0.00,New Mexico State,USA,DET,"6'4"" - 6'6"""
744,Zach Randolph,81.00,14.30,7.45,1.95,Michigan State,USA,SAC,"6'7"" - 6'9"""
745,Zaza Pachulia,83.00,5.13,4.83,1.60,None,Georgia,GSW,"6'10"" - 7'"
746,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI,"6'4"" - 6'6"""


In [29]:
final_merged_summary = final_merged_player_info_df.groupby(by = "Height Range").count()["height (in)"].to_frame()

final_merged_summary = final_merged_summary.rename(columns={'height (in)':'player_count'})
final_merged_summary

,player_count
Height Range,
<= 6ft,22
"6'1"" - 6'3""",109
"6'4"" - 6'6""",202
"6'7"" - 6'9""",233
"6'10"" - 7'",161
> 7ft,21


In [30]:
# Update the format of percentages in the following
pd.options.display.float_format = '{:.2f}%'.format

In [31]:
# Add percentages column to the df
count_players = len(players)
final_merged_summary['perc_of_players'] = final_merged_summary / count_players * 100
final_merged_summary

,player_count,perc_of_players
Height Range,,
<= 6ft,22,2.94%
"6'1"" - 6'3""",109,14.57%
"6'4"" - 6'6""",202,27.01%
"6'7"" - 6'9""",233,31.15%
"6'10"" - 7'",161,21.52%
> 7ft,21,2.81%


In [32]:
# Export final_merged_player_info_df 
player_output_path = os.path.join('Output_files_for_analysis', 'player_stats.csv')
final_merged_player_info_df.to_csv(player_output_path, header=True)

In [33]:
# Export final_nba_stats_with_sal_df
salary_output_path = os.path.join('Output_files_for_analysis', 'player_stats_with_sal.csv')
final_nba_stats_with_sal_df.to_csv(salary_output_path, header=True)

In [34]:
# Export organized_stats_df
stats_breakout_by_season_output_path = os.path.join('Output_files_for_analysis', 'player_stats_breakout_by_season.csv')
organized_stats_df.to_csv(stats_breakout_by_season_output_path, header=True)